In [ ]:
# importing
from keras.models import Model
from keras.engine.input_layer import Input
from keras.layers.core import Activation, Dense
from keras.layers.merge import concatenate
from keras.engine.input_layer import Input
from keras.layers import Flatten, Reshape,LSTM, Dropout
from tensorflow.keras.optimizers import Adam, RMSprop
from keras.models import load_model

In [ ]:
msg_size = 16
key_size = 16
ciper_size = 16


In [ ]:
# making models

# model for Alice
aliceInput0 = Input(shape=(msg_size,)) #message
aliceInput1 = Input(shape=(key_size,)) #key
aliceInput = concatenate([aliceInput0, aliceInput1], axis=1)

aliceDL1 = Dense(units=(msg_size + key_size))(aliceInput)
aliceDL1_activationFun = Activation('tanh')(aliceDL1)
aliceReshape = Reshape((msg_size + key_size, 1,))(aliceDL1_activationFun)

x = LSTM(16,return_sequences=True)(aliceReshape)
x = LSTM(64,return_sequences=True)(x)
x = LSTM(128)(x)
x = Dense(4096,activation='relu')(x)
x = Dropout(0.5)(x)
aliceOutput = Dense(16,activation='relu')(x)
alice = Model([aliceInput0, aliceInput1], aliceOutput, name='alice')
alice.summary()



# for bob

bobInput0 = Input(shape=(ciper_size,)) #message
bobInput1 = Input(shape=(key_size,)) #key
bobInput = concatenate([bobInput0, bobInput1], axis=1)

bobDL1 = Dense(units=(ciper_size + key_size))(bobInput)
bobDL1_activationFun = Activation('tanh')(bobDL1)
bobReshape = Reshape((msg_size + key_size, 1,))(bobDL1_activationFun)

x = LSTM(16,return_sequences=True)(bobReshape)
x = LSTM(64,return_sequences=True)(x)
x = LSTM(128)(x)
x = Dense(4096,activation='relu')(x)
x = Dropout(0.5)(x)
bobOutput = Dense(16,activation='relu')(x)
bob = Model([bobInput0, bobInput1], bobOutput, name='bob')
bob.summary()

# model for eve

eveInput = Input(shape=(ciper_size,)) #ciphertext only

eveDL1 = Dense(units=(ciper_size + key_size))(eveInput)
eveDL1_activationFun = Activation('tanh')(eveDL1)

eveDL2 = Dense(units=(ciper_size + key_size))(eveDL1_activationFun)
eveDL2_activationFun = Activation('tanh')(eveDL2)

eveReshape = Reshape((ciper_size + key_size, 1,))(eveDL2_activationFun)


x = LSTM(16,return_sequences=True)(eveReshape)
x = LSTM(64,return_sequences=True)(x)
x = LSTM(128)(x)
x = Dense(32,activation='relu')(x)
x = Dropout(0.5)(x)
eveOutput = Dense(16,activation='relu')(x)
eve = Model(eveInput, eveOutput, name='eve')
eve.summary()

Model: "alice"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_32 (InputLayer)          [(None, 16)]         0           []                               
                                                                                                  
 input_33 (InputLayer)          [(None, 16)]         0           []                               
                                                                                                  
 concatenate_14 (Concatenate)   (None, 32)           0           ['input_32[0][0]',               
                                                                  'input_33[0][0]']               
                                                                                                  
 dense_50 (Dense)               (None, 32)           1056        ['concatenate_14[0][0]']     

In [ ]:
from keras.models import Sequential

# alice model

Alice = Sequential()

aliceInput0 = Input(shape=(msg_size,)) #message
aliceInput1 = Input(shape=(key_size,)) #key
aliceInput = concatenate([aliceInput0, aliceInput1], axis=1)

Alice.add(Dense(units=(msg_size + key_size)))
Alice.add(Activation('tanh'))
Alice.add(Reshape((msg_size + key_size, 1,)))

Alice.add(LSTM(16,return_sequences=True))
Alice.add(LSTM(64,return_sequences=True))
Alice.add(LSTM(128))
Alice.add(Dense(4096,activation='tanh'))
Alice.add(Dropout(0.5))
Alice.add(Dense(16,activation='sigmoid'))

Alice.compile(loss = 'mse', optimizer = 'sgd', metrics = ['accuracy'])

Alice.build(aliceInput0.shape)

Alice.summary()

# Bob model

Bob = Sequential()

bobInput0 = Input(shape=(ciper_size,)) #cipher
bobInput1 = Input(shape=(key_size,)) #key
bobInput = concatenate([bobInput0, bobInput1], axis=1)

Bob.add(Dense(units=(ciper_size + key_size)))
Bob.add(Activation('tanh'))
Bob.add(Reshape((ciper_size + key_size, 1,)))

Bob.add(LSTM(16,return_sequences=True))
Bob.add(LSTM(64,return_sequences=True))
Bob.add(LSTM(128))
Bob.add(Dense(4096,activation='tanh'))
Bob.add(Dropout(0.5))
Bob.add(Dense(16,activation='sigmoid'))

Bob.compile(loss = 'mse', optimizer = 'sgd')

Bob.build(bobInput0.shape)

Bob.summary()

# Eve Model

Eve = Sequential()

eveInput0 = Input(shape=(ciper_size,)) #chiper

Eve.add(Dense(units=(ciper_size + key_size)))
Eve.add(Activation('tanh'))
Eve.add(Reshape((ciper_size + key_size, 1,)))

Eve.add(LSTM(16,return_sequences=True))
Eve.add(LSTM(64,return_sequences=True))
Eve.add(LSTM(128))
Eve.add(Dense(32,activation='tanh'))
Eve.add(Dropout(0.5))
Eve.add(Dense(16,activation='sigmoid'))

Eve.compile(loss = 'mse', optimizer = 'sgd')

Eve.build(eveInput0.shape)

Eve.summary()



Model: "sequential_56"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_192 (Dense)           (None, 32)                544       
                                                                 
 activation_66 (Activation)  (None, 32)                0         
                                                                 
 reshape_63 (Reshape)        (None, 32, 1)             0         
                                                                 
 lstm_189 (LSTM)             (None, 32, 16)            1152      
                                                                 
 lstm_190 (LSTM)             (None, 32, 64)            20736     
                                                                 
 lstm_191 (LSTM)             (None, 128)               98816     
                                                                 
 dense_193 (Dense)           (None, 4096)            

In [ ]:
print(eveInput0.shape)

(None, 16)


In [ ]:
# loss funs
from keras import backend as K

BOBINPUT1 = 1

aliceout = alice([aliceInput0, aliceInput1])
bobout = bob( [aliceout, bobInput1] )# bob sees ciphertext AND key
eveout = eve( aliceout )# eve doesn't see the key, only the cipher

eveloss = K.mean(  K.sum(K.abs(aliceInput0 - eveout), axis=-1)  )
bobloss = K.mean(  K.sum(K.abs(aliceInput0 - bobout), axis=-1)  )
abeloss = bobloss + K.square(msg_size/2 - eveloss)/( (msg_size//2)**2 )

Ayush = []
ind = 0
def abeloss1(ypred,ytrue):
  global ind
  aliceInput0, aliceInput1, bobInput1 = Ayush[0][ind],Ayush[1][ind],Ayush[2][ind]
  aliceout = alice([aliceInput0, aliceInput1])
  bobout = bob( [aliceout, bobInput1] )# bob sees ciphertext AND key
  eveout = eve( aliceout )# eve doesn't see the key, only the cipher

  eveloss = K.mean(  K.sum(K.abs(aliceInput0 - eveout), axis=-1)  )
  bobloss = K.mean(  K.sum(K.abs(aliceInput0 - bobout), axis=-1)  )
  abeloss = bobloss + K.square(msg_size/2 - eveloss)/( (msg_size//2)**2 )
  ind += 1
  if(ind == size(Ayush[0])):
    ind = 0
  return abeloss


abeoptim = RMSprop(lr=0.001)
eveoptim = Adam()





# Alice Bob model
AliceBob = Sequential()

AlicebobInput0 = Input(shape=(ciper_size,)) #cipher0
AlicebobInput1 = Input(shape=(key_size,)) #key
AlicebobInput = concatenate([bobInput0, bobInput1], axis=1)

AliceBob.add(Dense(units=(ciper_size + key_size + key_size )))# todo 3rd arg key_size
AliceBob.add(Activation('tanh'))
AliceBob.add(Reshape((ciper_size + key_size + key_size, 1,)))

AliceBob.add(LSTM(16,return_sequences=True))
AliceBob.add(LSTM(64,return_sequences=True))
AliceBob.add(LSTM(128))
AliceBob.add(Dense(4096,activation='tanh'))
AliceBob.add(Dropout(0.5))
AliceBob.add(Dense(16,activation='sigmoid'))
# AliceBob.add_loss(abeloss)
AliceBob.compile(loss = abeloss1, optimizer =  RMSprop(lr=0.001), metrics = ['accuracy'])

AliceBob.build(AlicebobInput0.shape)

AliceBob.summary()

# ///////////////////////Dummy eve

DummyEve = Sequential()

DummyeveInput0 = Input(shape=(ciper_size,)) #chiper

DummyEve.add(Dense(units=(ciper_size + key_size)))
DummyEve.add(Activation('tanh'))
DummyEve.add(Reshape((ciper_size + key_size, 1,)))

DummyEve.add(LSTM(16,return_sequences=True))
DummyEve.add(LSTM(64,return_sequences=True))
DummyEve.add(LSTM(128))
DummyEve.add(Dense(32,activation='tanh'))
DummyEve.add(Dropout(0.5))
DummyEve.add(Dense(16,activation='sigmoid'))
DummyEve.add_loss(eveloss)
DummyEve.compile(loss = 'mse', optimizer = Adam())



DummyEve.build(DummyeveInput0.shape)

DummyEve.summary()

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


Model: "sequential_63"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_213 (Dense)           (None, 48)                816       
                                                                 
 activation_73 (Activation)  (None, 48)                0         
                                                                 
 reshape_70 (Reshape)        (None, 48, 1)             0         
                                                                 
 lstm_210 (LSTM)             (None, 48, 16)            1152      
                                                                 
 lstm_211 (LSTM)             (None, 48, 64)            20736     
                                                                 
 lstm_212 (LSTM)             (None, 128)               98816     
                                                                 
 dense_214 (Dense)           (None, 4096)            

In [ ]:
import numpy as np
m_bits = msg_size
k_bits = key_size
batch_size = 512
m_batch = np.random.randint(0, 2, m_bits * batch_size).reshape(batch_size, m_bits)
k_batch = np.random.randint(0, 2, k_bits * batch_size).reshape(batch_size, k_bits)
# m_batch = m_batch * 2 - 1
# k_batch = k_batch * 2 -1
# concatenate([aliceInput0, aliceInput1], axis=1)
m_enc = AliceBob.predict([m_batch,k_batch],None)
m_enc*=2
m_enc = m_enc.astype(int)
m_enc = m_enc.astype(float)
print(type(m_enc), type(m_batch))
a=10
Ayush = [m_batch,k_batch,k_batch]
m_batch = np.random.randint(0, 2, 24 * batch_size).reshape(batch_size, 24)
AliceBob.fit(m_batch, m_enc, epochs=a, batch_size=a)

In [ ]:
result_train = Alice.evaluate(m_batch, m_enc)
print(result_train)

16/16 [==============================] - 0s 20ms/step - loss: 0.2372 - accuracy: 0.0000e+00
[0.2371748685836792, 0.0]


In [ ]:
print(m_batch[0],m_enc[0])

[1 0 1 1 0 0 1 1 0 1 1 1 1 1 1 0] [1. 0. 0. 0. 0. 1. 1. 0. 1. 1. 1. 0. 1. 0. 0. 1.]


In [ ]:
print(m_enc[0],m_enc[1],m_enc[2])

[0.501909   0.49750596 0.5035362  0.5017295  0.5003196  0.49937254
 0.49972373 0.49397856 0.49880672 0.4966245  0.5046972  0.5021803
 0.49604765 0.5020065  0.50044376 0.5020706 ] [0.4988807  0.50139934 0.49710816 0.4983344  0.49940416 0.49968562
 0.50058717 0.5037307  0.5006836  0.50045574 0.49731314 0.49903655
 0.50160116 0.49871123 0.49980858 0.49769408] [0.50039124 0.49972025 0.499817   0.49961835 0.5000808  0.4998048
 0.5000748  0.4994498  0.49981362 0.4989307  0.500539   0.5007037
 0.49938425 0.5001561  0.5002254  0.49976435]


In [ ]:
epochs = 20
batch_size = 512
m_train = 2<<msg_size
batch_count = m_train//batch_size

G_aliceBob_loss = []
G_bob_loss = []
G_eve_loss = []

for epoch in range(epochs):
  aliceBob_loss = []
  bob_loss = []
  eve_loss = []

  for i in range(batch_count):
    Alice.trainable = True
    MSG = np.random.randint(0,2,msg_size*batch_size).reshape(batch_size,msg_size)
    KEY = np.random.randint(0,2,key_size*batch_size).reshape(batch_size,key_size)
    loss = AliceBob.train_on_batch([MSG,KEY])

    G_aliceBob_loss.append(loss)
    aliceBob_loss.append(loss)
    avg_aliceBob = np.mean(aliceBob_loss)

    alicePred = Alice.predict([MSG,KEY])

    bobPred = Bob.predict([alicePred,KEY])

    loss = np.mean(  np.sum( np.abs(MSG - bobPred), axis=-1)  )

    bob_loss.append(loss)
    G_bob_loss.append(loss)
    avg_Bob = np.mean(bob_loss)

    alice.trainable = False

    for i in range(2):
      MSG = np.random.randint(0,2,msg_size*batch_size).reshape(batch_size,msg_size)
      KEY = np.random.randint(0,2,key_size*batch_size).reshape(batch_size,key_size)
      loss = DummyEve.train_on_batch([MSG,KEY],None)
    
    G_eve_loss.append(loss)
    eve_loss.append(loss)
    avg_eve = np.mean(eve_loss)

    if i==batch_count-1:
      print(epoch, avg_aliceBob, avg_Bob, avg_eve)


print('Training finished.')

    


TypeError: ignored